The parser class 

In [204]:


from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON

 
sparql_query='SELECT DISTINCT ?uri WHERE { ?x <http://dbpedia.org/property/international> <http://dbpedia.org/resource/Muslim_Brotherhood> . ?x <http://dbpedia.org/ontology/religion> ?uri  . ?x <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/PoliticalParty> . } '
a= SPARQL(sparql_query)



class Triples: # chaneg name
        def __init__(self, triple_list):
            self.triple_list = triple_list
        
        def get_triple_list(self):
            return self.triple_list
        
        def get_subject(self):
            return self.triple_list[0]
        
        def get_predicate(self):
            return self.triple_list[1]
        
        def _get_object(self):
            return self.triple_list[2]
        
        


triples_list= []


for s in a.triple_list:
    triples_list.append(Triples(s.split(' ')))

for t in triples_list:
    # print(t.get_triple_list())
    print("sub " + t.get_subject())
    print("pre " + t.get_predicate())
    print("obj " + t._get_object())
    print('------------------')






sub ?x
pre <http://dbpedia.org/property/international>
obj <http://dbpedia.org/resource/Muslim_Brotherhood>
------------------
sub ?x
pre <http://dbpedia.org/ontology/religion>
obj ?uri
------------------
sub ?x
pre <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>
obj <http://dbpedia.org/ontology/PoliticalParty>
------------------


The endpoint 

In [188]:
sparql = SPARQLWrapper("https://api.triplydb.com/datasets/academy/pokemon/services/pokemon/sparql")
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()  

index = results['head']['vars']

result_list = []
ind_reslut = []


for y in index:
    for x in results['results']['bindings']:
        ind_reslut.append(x[y]['value'])
    result_list.append(ind_reslut)
    ind_reslut = []
print(result_list)


[['https://triplydb.com/academy/pokemon/vocab/ability', 'https://triplydb.com/academy/pokemon/vocab/effortAttack', 'https://triplydb.com/academy/pokemon/vocab/baseAttack', 'https://triplydb.com/academy/pokemon/vocab/baseDefense', 'https://triplydb.com/academy/pokemon/vocab/baseExp', 'https://triplydb.com/academy/pokemon/vocab/baseHP', 'https://triplydb.com/academy/pokemon/vocab/baseSpAtk', 'https://triplydb.com/academy/pokemon/vocab/baseSpDef', 'https://triplydb.com/academy/pokemon/vocab/baseSpeed', 'https://triplydb.com/academy/pokemon/vocab/colour'], ['Ability', 'Attack effort', 'Base attack', 'Base defense statistic', 'Base experience', 'Base health', 'Base special attack', 'Base special defense', 'Base speed', 'Color']]


In [ ]:
def answer_convert(item_answer):
    if 'boolean' in item_answer.keys():
        at = 'boolean'
    else:
        at = item_answer['head']['vars'][0]
    answer = []
    if at == 'boolean':
        answer.append(item_answer['boolean'])
    else:
        for cand in item_answer['results']['bindings']:
            if at == 'date':
                answer.append(cand['date']['value'])
            elif at == 'number':
                answer.append(cand['c']['value'])
            elif at == 'resource' or at == 'uri':
                answer.append(cand['uri']['value'])
            elif at == 'string':
                answer.append(cand['string']['value'])
            elif at == 'callret-0':
                answer.append(cand['callret-0']['value'])
            else: 
                answer.append(cand[at]['value'])
    return answer


sparql = SPARQLWrapper(
    "https://api.triplydb.com/datasets/academy/pokemon/services/pokemon/sparql")
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()  
answer = answer_convert(results)

type(results.items())

print(results.keys())
print('boolean' in results.keys())
